# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [78]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import ipywidgets as widgets

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [85]:
cities = pd.read_csv(os.path.join("..","WeatherPy","output_data","cities.csv"))
print(cities.dtypes)
cities["Humidity"].max()

Unnamed: 0      int64
Name           object
Cloudiness      int64
Country        object
Date            int64
Humidity        int64
Lat           float64
Lng           float64
Max Temp      float64
Wind Speed    float64
dtype: object


100

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [86]:
gmaps.configure(api_key=g_key)

In [87]:
pic = gmaps.figure()
heatLayer = gmaps.heatmap_layer(cities[["Lat", "Lng"]], weights=cities["Humidity"],max_intensity=cities["Humidity"].max(), point_radius=1, dissipating=False)
pic.add_layer(heatLayer)
pic

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [22]:
ideal_places_df = cities.loc[(cities["Max Temp"] > 68) & 
                             (cities["Max Temp"] < 87) & 
                             (cities["Cloudiness"] == 0) & 
                             (cities["Wind Speed"] < 5)]

print(ideal_places_df["Max Temp"].max())
print(ideal_places_df["Max Temp"].min())
print(ideal_places_df["Cloudiness"].max())
print(ideal_places_df["Cloudiness"].min())
print(ideal_places_df["Wind Speed"].max())
print(ideal_places_df["Wind Speed"].min())
print(ideal_places_df.count())
# no need to use dropna() because there are no null values

83.08
71.6
0
0
4.03
1.36
Unnamed: 0    7
Name          7
Cloudiness    7
Country       7
Date          7
Humidity      7
Lat           7
Lng           7
Max Temp      7
Wind Speed    7
dtype: int64


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [50]:
hotel_df = ideal_places_df
hotel_df["Hotel Name"] = ""

hotel_df

/home/mario/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Unnamed: 0,Name,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
11,11,Rikitea,0,PF,1589317468,66,-23.12,-134.97,74.25,2.59,
15,15,Abu Kamal,0,SY,1589317469,27,34.45,40.92,77.54,2.30,
113,113,Bilma,0,NE,1589317493,13,18.69,12.92,82.81,1.83,
133,133,Faya,0,SA,1589317498,73,18.39,42.45,71.60,1.36,
401,401,Nizwá,0,OM,1589317566,21,22.93,57.53,83.08,2.64,
413,413,Rodolfo Sánchez Taboada,0,MX,1589317569,72,31.72,-116.57,72.00,4.00,
519,519,Antsohihy,0,MG,1589317593,81,-14.88,47.99,73.60,4.03,


In [51]:
parameters = {
              "radius":5000,
              "keyword":"hotel",
              "types":"hotel",
              "key":g_key
             }

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [71]:
for idx, row in hotel_df.iterrows():
        parameters["location"] = f'{row["Lat"]},{row["Lng"]}'
        response = requests.get(url, parameters).json()
        
        if response['status'] == "OK":
             hotel_df.at[idx,'Hotel Name'] = response['results'][0]['name']
        else:
            hotel_df.at[idx,'Hotel Name']= "Not Found"

In [72]:
hotel_df


,Unnamed: 0,Name,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
11,11,Rikitea,0,PF,1589317468,66,-23.12,-134.97,74.25,2.59,Pension Maro'i
15,15,Abu Kamal,0,SY,1589317469,27,34.45,40.92,77.54,2.30,Not Found
113,113,Bilma,0,NE,1589317493,13,18.69,12.92,82.81,1.83,Not Found
133,133,Faya,0,SA,1589317498,73,18.39,42.45,71.60,1.36,Not Found
401,401,Nizwá,0,OM,1589317566,21,22.93,57.53,83.08,2.64,Al Diyar Hotel
413,413,Rodolfo Sánchez Taboada,0,MX,1589317569,72,31.72,-116.57,72.00,4.00,Not Found
519,519,Antsohihy,0,MG,1589317593,81,-14.88,47.99,73.60,4.03,Hotel Belle Vue


In [75]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [77]:
# Add marker layer ontop of heat map
marks = gmaps.marker_layer(locations)
pic.add_layer(marks)
pic

# Display Map

Figure(layout=FigureLayout(height='420px'))